In [2]:
# | default_exp compute_tleaf

In [3]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
import numpy as np

In [3]:
# | export

def compute_tleaf(t_air:float, # Air temperature (degC)
                  par:float, # Unknown parameter definition
                  potential_par:float, # Unknown parameter definition Calculated using `potential_par` function?
                  wind_speed:float, # Unknown parameter definition (m/s)
                  relative_humidity:int, # Air relative_humidity(%)
                  gs:float, # Stomatal conductance
                  g_cuti:float, # leaf conductance
                  e_inst:float, # Unknown parameter definition
                  psi_leaf:float, # Unknown parameter definition
                  leaf_size:float = 50, # Characteristic dimension from vegetation params in mm i.e. 1 (pine needle) to 3000 (banana leaf)
                  leaf_angle:int = 45, # Leaf angle (depuis le plan horizontal : 0-90 deg)
                  turn_off_eb:bool = False, # Unknown parameter definition
                  transpiration_model: str = ["jarvis", "granier"], # Transpiration model type
                  ) -> float:

        "Compute leaf temperature and Vapour Pressure deficit"

        # Assert parameters -----------------------------------------------------
        assert  0 <= relative_humidity <= 100, 'relative_humidity must be a value between 0 and 100'

        assert  0 <= leaf_angle <= 90, 'leaf_angle must be a value between 0 and 90'

        assert transpiration_model in [
        "jarvis",
        "granier",
        ], f'{transpiration_model} not a valid option, choose  "jarvis" or "granier"'


In [2]:
0 <= 50

True